## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickets of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## collect index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

rho=0.005

In [5]:
tickind

[315, 216, 372, 388]

# only use 4 stocks

In [4]:
################## MODEL I ##################
mod = Model()

#Define Decision Variables
w = mod.addVars(4)

#Add constraints
mod.addConstr(sum (w[i] * avg_return[tickind][i] for i in range(4)) >= rho)

mod.addConstr(sum (w[i] for i in range(4)) == 1)

#Non-negativity
mod.addConstrs(w[i] >= 0.0 for i in range(4))


# Create the objective function, and set it to be minimized
mod.setObjective(sum (sum(w[i] * w[j] * C[tickind].T[tickind][i,j] for i in range(4) ) for j in range (4)),GRB.MINIMIZE) 

Using license file /Users/wjq/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-12


In [5]:
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x70c79b8a
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+

In [6]:
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
w_opt = [w[i].x for i in range(4)]

for i in range(4):
    print(f'{tick4[i]}',w_opt[i])

opt_val = mod.objval
print("Optimal value:",opt_val)

Solved to optimality
MSFT 0.2371174913710205
GS 0.02585983533068362
PG 1.5749710132197093e-09
SCHP 0.7370226717232978
Optimal value: 0.0001774932651657826


# use all the available stocks

In [7]:
################## MODEL II ##################
mod = Model()

#Define Decision Variables
w = mod.addVars(n)

#Add constraints
mod.addConstr(sum (w[i] * avg_return[i] for i in range(n)) >= rho)

mod.addConstr(sum (w[i] for i in range(n)) == 1)

#Non-negativity
mod.addConstrs(w[i] >= 0.0 for i in range(n))


# Create the objective function, and set it to be minimized
mod.setObjective(sum (sum(w[i] * w[j] * C[i,j] for i in range(n) ) for j in range (n)),GRB.MINIMIZE) 

In [8]:
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xd3a53ccf
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s


In [9]:
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
w_opt = [w[i].x for i in range(n)]

for i in range(n):
    print(f'{tickers[i]}',w_opt[i])

opt_val = mod.objval
print("Optimal value:",opt_val)

Solved to optimality
MMM 4.107560039965785e-09
ABT 4.687773573635427e-09
ABBV 0.011356491596383177
ABMD 0.0066638570304316065
ACN 6.887442113747223e-09
ATVI 0.01368735889883513
ADBE 7.992051298216673e-09
AMD 1.639084653014167e-09
AAP 4.132339125545912e-09
AES 7.145953772608971e-09
AMG 2.6565577175574394e-09
AFL 6.770819508625927e-09
A 3.6634320568808067e-09
APD 5.614952414625803e-09
AKAM 5.081016406463897e-09
ALK 3.6170282965200505e-09
ALB 5.141150778130226e-09
ARE 4.625576372930741e-09
ALXN 4.8615142840579696e-09
ALGN 3.3501855323756885e-09
ALLE 3.145686872921073e-09
AGN 2.8063321986829254e-09
ADS 4.046241633510747e-09
LNT 1.0269350621216422e-08
ALL 5.702235708454717e-09
GOOGL 5.573279843776687e-09
GOOG 5.48285102130117e-09
MO 6.437561950834866e-09
AMZN 1.8448234621564803e-08
AMCR 4.817666615908616e-09
AEE 1.2122502666071613e-08
AAL 1.6388283794725243e-09
AEP 1.1543873432864412e-08
AXP 1.0268619231217037e-08
AIG 1.5730319741931394e-08
AMT 4.95574520863153e-09
AWK 6.409270187636447e-08

# all the stocks should be considered but not necessarily be used

In [10]:
################## MODEL III ##################
mod2 = Model()

#Define Decision Variables
w2 = mod2.addVars(n)
x2 = mod2.addVars(n, vtype = GRB.BINARY)

#Add constraints
mod2.addConstr(sum (x2[i] for i in range(n)) <= 4)

mod2.addConstr(sum (w2[i] * x2[i] * avg_return[i] for i in range(n)) >= rho)

mod2.addConstr(sum (w2[i] * x2[i] for i in range(n)) == 1)

mod2.addConstrs(x2[i] >= w2[i] for i in range(n))

#Non-negativity
mod2.addConstrs(w2[i] >= 0.0 for i in range(n))

# Create the objective function, and set it to be minimized
mod2.setObjective(sum (sum(w2[i] * w2[j] * C[i,j] for i in range(n) ) for j in range (n)),GRB.MINIMIZE) 

In [11]:
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 781 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xca157c2a
Model has 76245 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
  QRHS range       [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 9521 iterations, 0.65 seconds

    Nodes    |    Current Node    |     Obj

In [12]:
if mod2.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
w2_opt = [w2[i].x for i in range(n)]

for i in range(n):
    if w2_opt[i] > 0:
        print(f'{tickers[i]}',w2_opt[i])

opt_val = mod2.objval
print("Optimal value:",opt_val)

Solved to optimality
CME 0.1264114192949047
LLY 0.07547619035437458
NVDA 0.043753706728431804
BND 0.754358683622289
Optimal value: 6.753470760728118e-05


# set a maximum time limit

In [13]:
mod2.Params.TimeLimit = 30.0

mod2.update()
mod2.optimize()

if mod2.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
w2_opt = [w2[i].x for i in range(n)]

for i in range(n):
    if w2_opt[i] > 0:
        print(f'{tickers[i]}',w2_opt[i])

opt_val = mod2.objval
print("Optimal value:",opt_val)

Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 781 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xca157c2a
Model has 76245 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
  QRHS range       [5e-03, 1e+00]
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms

Continuing optimization...


Cutting planes:
  Cover: 1
  Implied bound: 1
  MIR: 20
  Flow cover: 14
  Relax-and-lift: 1

Explored 61490 nodes (3502684 simplex iterations) in 0.02 second

# set a maximum acceptable optimality gap (the tolerance)

In [18]:
mod2 = Model()

#Define Decision Variables
w2 = mod2.addVars(n)
x2 = mod2.addVars(n, vtype = GRB.BINARY)

#Add constraints
mod2.addConstr(sum (x2[i] for i in range(n)) <= 4)

mod2.addConstr(sum (w2[i] * x2[i] * avg_return[i] for i in range(n)) >= rho)

mod2.addConstr(sum (w2[i] * x2[i] for i in range(n)) == 1)

mod2.addConstrs(x2[i] >= w2[i] for i in range(n))

#Non-negativity
mod2.addConstrs(w2[i] >= 0.0 for i in range(n))

# Create the objective function, and set it to be minimized
mod2.setObjective(sum (sum(w2[i] * w2[j] * C[i,j] for i in range(n) ) for j in range (n)),GRB.MINIMIZE) 

mod2.Params.MIPGap = 0.1

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001


In [19]:
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 781 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xca157c2a
Model has 76245 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
  QRHS range       [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 9521 iterations, 0.68 seconds

    Nodes    |    Current Node    |     Obj

In [20]:
if mod2.status == GRB.OPTIMAL:
    print("Solved to optimality")
    
w2_opt = [w2[i].x for i in range(n)]

for i in range(n):
    if w2_opt[i] > 0:
        print(f'{tickers[i]}',w2_opt[i])

opt_val = mod2.objval
print("Optimal value:",opt_val)

Solved to optimality
CME 0.1264114192949047
LLY 0.07547619035437458
NVDA 0.043753706728431804
BND 0.754358683622289
Optimal value: 6.753470760728118e-05
